# <center> <font style="color:rgb(100,109,254)"> Playing Rock, Paper, Scissors with AI</font> </center>


In [55]:
# !jt -l

In [56]:
# !jt -t chesterish -T -N -kl
# !jt -r

###  <font style="color:rgb(134,19,348)">Import Libraries </font>

In [57]:
import os
import cv2
import time
import numpy as np

from scipy import stats as st
from random import choice
from collections import deque
from tensorflow.keras.models import load_model

### Declare global variables

In [58]:
# Initialize the camera
cap = cv2.VideoCapture(0)

# Frame from camera video
frame = None

# Trigger tells us when to start recording
trigger = False

# hand_sign to hold name of each of hand sign.
hand_sign = ''

# Counter keeps count of the number of samples collected
counter = 0

# Indicator text on screen
text = ''

# This the ROI size, the size of images saved will be box_size - 10 (= 224)
box_size = 234

# Getting the width of the frame from the camera properties
width = int(cap.get(3))

# Specify the number of attempts you want.
attempts = 10

In [ ]:
# Initially the moves will be `nothing`
computer_move_name= "nothing"
final_user_move = "nothing"

# All scores are 0 at the start.
computer_score, user_score = 0, 0

# The default color of bounding box is Blue
rect_color = (255, 0, 0)

# This variables remembers if the hand is inside the box or not.
hand_inside = False

# At each iteration we will decrease the total_attempts value by 1
total_attempts = attempts

# We will only consider predictions having confidence above this threshold.
confidence_threshold = 0.70

# Instead of working on a single prediciton, we will take the mode of 5 predictons by using a deque object
# This way even if we face a false postive, we would easily ignore it
smooth_factor = 5

# Our initial deque list will have 'nothing' repeated 5 times.
de = deque(['nothing'] * 5, maxlen=smooth_factor)

final_user_move, computer_move_name, rect_color, winner = '', '', None, ''

### Create UI function

In [59]:
def create_ui_pro(title):
    global frame
    
    # Flip the frame to make it more realistic.
    frame = cv2.flip(frame, 1)

    # Define ROI for capturing samples
    cv2.rectangle(frame, (width - box_size, 0), (width, box_size), (0, 250, 150), 2)

    # Make a resizable window.
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)

In [60]:
def show_ui(title):
    global frame, text
    
    # Show the counter on the imaege
    cv2.putText(frame, text, (3, 350), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 1, cv2.LINE_AA)

    # Display the window
    cv2.imshow(title, frame)

###  <font style="color:rgb(134,19,348)"> Load the model </font>

In [61]:
model = load_model("rps4.h5")

### AI model prediction function

In [62]:
# Name of hand signs
hand_signs_list = ['nothing', 'paper', 'rock', 'scissor']

In [63]:
def make_prediction():
    global frame
    roi = frame[5: box_size-5 , width-box_size + 5: width -5]
    
    # Normalize the image like we did in the preprocessing step, also convert float64 array.
    roi = np.array([roi]).astype('float64') / 255.0
 
    # Get model's prediction.
    pred = model.predict(roi)
    
    # Get the index of the target class.
    move_code = np.argmax(pred[0])
    
    # Get the class name of the predicted class
    user_move = hand_signs_list[move_code]

    # Get the probability of the target class
    prob = np.max(pred[0])
    
    return pred, user_move, prob

###  <font style="color:rgb(134,19,348)"> Function for finding the Winner </font>
First we need a function that takes two moves, one by the user and one the computer and then finds out who won that round. This function is pretty simple as you can see below.

In [64]:
def findout_winner_one(user_move, Computer_move):
    
    # All logic below is self explanatory 
    
    if user_move == Computer_move:
        return "Tie"
    
    
    elif user_move == "rock" and Computer_move == "scissor":
        return "User"
    
    elif user_move == "rock" and Computer_move == "paper":
        return "Computer"
    
    elif user_move == "scissor" and Computer_move == "rock":
        return "Computer"
    
    elif user_move == "scissor" and Computer_move == "paper":
        return "User"
    
    elif user_move == "paper" and Computer_move == "rock":
        return "User"
    
    elif user_move == "paper" and Computer_move == "scissor":
        return "Computer"

###  <font style="color:rgb(134,19,348)"> Function for find the image of Winner at the end Game </font>

In [65]:
def get_winner_img():    
    if user_score > computer_score:
        winner_img = 'youwin.jpg'
        
    elif user_score < computer_score:
        winner_img = 'comwin.jpg'
        
    else:
        winner_img = 'nobodywin.jpg'
        
    return winner_img

###  <font style="color:rgb(134,19,348)"> Function for Displaying Computer's Move (Optional) </font>

This function displays a picture of either 'rock', 'paper' or  'scissor' based on what the computer played. 

In [66]:
def display_computer_move(computer_move_name):
    global frame
    icon = cv2.imread( "images/{}.png".format(computer_move_name), 1)
    icon = cv2.resize(icon, (224,224))
    
    # This is the portion which we are going to replace with the icon image
    roi = frame[0:224, 0:224]

    # Get binary mask from the transparent image, 4th channel is the alpha channel 
    mask = icon[:,:,-1] 

    # Making the mask completely binary (black & white)
    mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)[1]

    # Store the normal bgr image
    icon_bgr = icon[:,:,:3] 
    
    # Now combine the foreground of the icon with background of ROI 
    
    img1_bg = cv2.bitwise_and(roi, roi, mask = cv2.bitwise_not(mask))

    img2_fg = cv2.bitwise_and(icon_bgr, icon_bgr, mask = mask)

    combined = cv2.add(img1_bg, img2_fg)

    frame[0:224, 0:224] = combined

    return frame

### Function to display image of winner at the end game

In [67]:
def display_winner(winner):
    global frame
    icon = cv2.imread( "images/{}".format(winner), 1)
    icon = cv2.resize(icon, (224,224))
    
    # This is the portion which we are going to replace with the icon image
    roi = frame[256:480, 208:432]

    # Get binary mask from the transparent image, 4th channel is the alpha channel 
    mask = icon[:,:,-1] 

    # Making the mask completely binary (black & white)
    mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)[1]

    # Store the normal bgr image
    icon_bgr = icon[:,:,:3] 
    
    # Now combine the foreground of the icon with background of ROI 
    
    img1_bg = cv2.bitwise_and(roi, roi, mask = cv2.bitwise_not(mask))

    img2_fg = cv2.bitwise_and(icon_bgr, icon_bgr, mask = mask)

    combined = cv2.add(img1_bg, img2_fg)

    frame[256:480, 208:432] = combined

### Function to calculate game score

In [68]:
def cal_score():
    global user_score, computer_score, total_attempts, rect_color, winner
    
    # Subtract one attempt
    total_attempts -= 1

    # If winner is computer then it gets points and vice versa.
    # We're also changing the color of rectangle based on who wins the round.

    if winner == "Computer":
        computer_score +=1
        rect_color = (0, 0, 255)

    elif winner == "User":
        user_score += 1;
        rect_color = (0, 250, 0)


    elif winner == "Tie":
        rect_color = (255, 250, 255)

### Function to display game indication text on screen

In [69]:
def show_ui_game():

    global rect_color, total_attempts, frame
    
    # This is where all annotation is happening. 
    
    cv2.putText(frame, "Your Score: " + str(user_score),
                    (450, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Computer Score: " + str(computer_score),
                    (2, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1, cv2.LINE_AA)

    
    if total_attempts != 0:
        cv2.putText(frame, "Your Move: " + final_user_move,
                    (450, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (8, 143, 143), 1, cv2.LINE_AA)

        cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (2, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (8, 143, 143), 1, cv2.LINE_AA)
        
        cv2.putText(frame, "Attempts left: {}".format(total_attempts), (190, 400), cv2.FONT_HERSHEY_COMPLEX, 
                    0.7, (100, 2, 255), 1, cv2.LINE_AA)
        
        cv2.rectangle(frame, (width - box_size, 0), (width, box_size), rect_color, 2) 

    else:
        cv2.putText(frame, 'Press ENTER to play again, q to exit.', (130, 450), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                (139, 0, 0), 2, cv2.LINE_AA)
        

    # Display the image    
    cv2.imshow("Rock Paper Scissors", frame)

    # Exit if 'q' is pressed 
    k = cv2.waitKey(10)
    if k == ord('q'):
        return False
    
    return True

### Function to start play game

In [70]:
# If nothing is not true and hand_inside is False then proceed.
# Basically the hand_inside variable is helping us to not repeatedly predict during the loop
# So now the user has to take his hands out of the box for every new predicton.
        
def play():
    global user_score, computer_score, total_attempts, computer_move_name, hand_inside, rect_color,
        winner, frame
    
    if final_user_move != "nothing" and hand_inside == False:

        # Set hand inside to True
        hand_inside = True 

        # Get Computer's move and then get the winner.
        computer_move_name = choice(['rock', 'paper', 'scissor'])
        winner = findout_winner_one(final_user_move, computer_move_name)

        # Display the computer's move
        display_computer_move(computer_move_name)

        cal_score()
        

    # Display images when the hand is inside the box even when hand_inside variable is True.
    elif final_user_move != "nothing" and hand_inside == True:
        display_computer_move(computer_move_name)

    # If class is nothing then hand_inside becomes False
    elif final_user_move == 'nothing':            
        hand_inside = False
        rect_color = (255, 0, 0)
    
    if total_attempts == 0:
        winner_img = get_winner_img()
        display_winner(winner_img)

        # If enter is pressed.
        k = cv2.waitKey(1)

        # If the user presses 'ENTER' key then return TRUE, otherwise FALSE
        if k == 13:
             user_score, computer_score, total_attempts = 0, 0, attempts

### Main flow game

In [71]:
while True:
    
    ret, frame = cap.read()
    
    if not ret:
        break
        
    create_ui_pro('Rock Paper Scissors')
    
    pred, user_move, prob = make_prediction()
    
    # Make sure the probability is above our defined threshold
    if prob >= confidence_threshold:
        
        # Now add the move to deque list from left
        de.appendleft(user_move)
        
        # Get the mode i.e. which class has occured more frequently in the last 5 moves.
        try:
            final_user_move = st.mode(de)[0][0] 
            
        except StatisticsError:
            print('Stats error')
            continue      
        play()

    if not show_ui_game():
        break

# Relase the camera and destroy all windows.
cap.release()
cv2.destroyAllWindows()